In [ ]:
import pandas as pd
import json
# import SPARQLWrapper
from SPARQLWrapper import SPARQLWrapper, JSON
import ast
from tqdm import tqdm


# import docx
from docx import Document
from tabulate import tabulate
from IPython.display import display, HTML
import matplotlib.backends.backend_pdf
import matplotlib.pyplot as plt

In [2]:
pd.set_option('max_rows', 600)
pd.set_option('max_colwidth', 300)

In [3]:
%store -r rankings
rankings

,statement,rank,score,probas_pos_neg
0,https://data.cooperationdatabank.org/vocab/prop/anonymityManipul_H1 https://data.cooperationdatabank.org/vocab/prop/hasPositiveEffectOn https://data.cooperationdatabank.org/id/dependentvariable/cooperation,1,7.823242,0.989597
1,https://data.cooperationdatabank.org/vocab/prop/sVOType_H1 https://data.cooperationdatabank.org/vocab/prop/hasPositiveEffectOn https://data.cooperationdatabank.org/id/dependentvariable/cooperation,1,7.789260,0.989320
2,https://data.cooperationdatabank.org/vocab/prop/emotion_H11 https://data.cooperationdatabank.org/vocab/prop/hasPositiveEffectOn https://data.cooperationdatabank.org/id/dependentvariable/withdrawals,1,7.646037,0.988068
3,https://data.cooperationdatabank.org/vocab/prop/positionInGame_H2 https://data.cooperationdatabank.org/vocab/prop/hasPositiveEffectOn https://data.cooperationdatabank.org/id/dependentvariable/withdrawals,1,7.612097,0.987750
4,https://data.cooperationdatabank.org/vocab/prop/iteratedStrategy_H6 https://data.cooperationdatabank.org/vocab/prop/hasPositiveEffectOn https://data.cooperationdatabank.org/id/dependentvariable/cooperation,1,7.601054,0.987645
...,...,...,...,...
1058,https://data.cooperationdatabank.org/vocab/prop/punishmentAgent_H3 https://data.cooperationdatabank.org/vocab/prop/hasNegativeEffectOn https://data.cooperationdatabank.org/id/dependentvariable/cooperation,1510,0.247662,0.201819
1059,https://data.cooperationdatabank.org/vocab/prop/punishmentAgent_H3 https://data.cooperationdatabank.org/vocab/prop/hasNegativeEffectOn https://data.cooperationdatabank.org/id/dependentvariable/withdrawals,2339,0.140320,0.188622
1060,https://data.cooperationdatabank.org/vocab/prop/partnerEmotion_H3 https://data.cooperationdatabank.org/vocab/prop/hasPositiveEffectOn https://data.cooperationdatabank.org/id/dependentvariable/cooperation,9372,-0.237753,0.147424
1061,https://data.cooperationdatabank.org/vocab/prop/communicationReal_H1 https://data.cooperationdatabank.org/vocab/prop/hasPositiveEffectOn https://data.cooperationdatabank.org/id/dependentvariable/withdrawals,10239,-0.260704,0.145181


In [4]:
url = 'https://api.cooperationdatabank.org/datasets/coda-dev/databank/services/databank/sparql'

def get_sparql_dataframe(service, query):
    sparql = SPARQLWrapper(service)
    out = []
   
    sparql.setQuery(query)

    sparql.setReturnFormat(JSON)
    result = sparql.query()
    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)
        
    return pd.DataFrame(out, columns=cols)

In [5]:
observations = pd.read_csv('data/observations.csv', index_col = 0)

In [6]:
observations

,obs,dependent,ES,N,t1,t2,study,design,n1,n2,...,ESLower,ESUpper,paper,doi,independentProperties,SE,categoryT1,categoryT2,IV_new,ESType
0,https://data.cooperationdatabank.org/id/ENG00008_1.4.1.2.d,https://data.cooperationdatabank.org/id/dependentvariable/contributions,1.243930,57.0,https://data.cooperationdatabank.org/id/ENG00008_1.4.1,https://data.cooperationdatabank.org/id/ENG00008_1.4.2,https://data.cooperationdatabank.org/id/ENG00008_1,https://data.cooperationdatabank.org/id/betweenorwithinparticipantsdesign/1,28.0,29.0,...,0.676655,1.811200,https://data.cooperationdatabank.org/id/ENG00008,http://dx.doi.org/10.1016/j.jesp.2004.09.004,['https://data.cooperationdatabank.org/vocab/prop/sequentiality'],0.264493,https://data.cooperationdatabank.org/id/sequentiality/simultaneous,https://data.cooperationdatabank.org/id/sequentiality/sequential_turn-taking,https://data.cooperationdatabank.org/vocab/prop/sequentiality_H1,LargeMediumPositiveES
1,https://data.cooperationdatabank.org/id/ENG00008_1.4.1.3.d,https://data.cooperationdatabank.org/id/dependentvariable/contributions,1.084430,57.0,https://data.cooperationdatabank.org/id/ENG00008_1.4.1,https://data.cooperationdatabank.org/id/ENG00008_1.4.3,https://data.cooperationdatabank.org/id/ENG00008_1,https://data.cooperationdatabank.org/id/betweenorwithinparticipantsdesign/1,28.0,29.0,...,0.528299,1.640570,https://data.cooperationdatabank.org/id/ENG00008,http://dx.doi.org/10.1016/j.jesp.2004.09.004,['https://data.cooperationdatabank.org/vocab/prop/sequentiality'],0.264041,https://data.cooperationdatabank.org/id/sequentiality/simultaneous,https://data.cooperationdatabank.org/id/sequentiality/sequential_turn-taking,https://data.cooperationdatabank.org/vocab/prop/sequentiality_H1,LargeMediumPositiveES
2,https://data.cooperationdatabank.org/id/ENG00008_1.4.2.3.d,https://data.cooperationdatabank.org/id/dependentvariable/contributions,-0.087300,58.0,https://data.cooperationdatabank.org/id/ENG00008_1.4.2,https://data.cooperationdatabank.org/id/ENG00008_1.4.3,https://data.cooperationdatabank.org/id/ENG00008_1,https://data.cooperationdatabank.org/id/betweenorwithinparticipantsdesign/1,29.0,29.0,...,-0.602256,0.427657,https://data.cooperationdatabank.org/id/ENG00008,http://dx.doi.org/10.1016/j.jesp.2004.09.004,['https://data.cooperationdatabank.org/vocab/prop/positionInGame'],0.262613,1,2,https://data.cooperationdatabank.org/vocab/prop/positionInGame_H1,NullFinding
3,https://data.cooperationdatabank.org/id/ENG00008_2.1.1.2.d,https://data.cooperationdatabank.org/id/dependentvariable/contributions,-0.324082,192.0,https://data.cooperationdatabank.org/id/ENG00008_2.1.1,https://data.cooperationdatabank.org/id/ENG00008_2.1.2,https://data.cooperationdatabank.org/id/ENG00008_2,https://data.cooperationdatabank.org/id/betweenorwithinparticipantsdesign/1,96.0,96.0,...,-0.608830,-0.039335,https://data.cooperationdatabank.org/id/ENG00008,http://dx.doi.org/10.1016/j.jesp.2004.09.004,['https://data.cooperationdatabank.org/vocab/prop/expectationsLevel'],0.144338,https://data.cooperationdatabank.org/id/expectationslevel/low,https://data.cooperationdatabank.org/id/expectationslevel/high,https://data.cooperationdatabank.org/vocab/prop/expectationsLevel_H1,SmallNegativeES
4,https://data.cooperationdatabank.org/id/ENG00008_2.2.1.2.d,https://data.cooperationdatabank.org/id/dependentvariable/contributions,0.315689,144.0,https://data.cooperationdatabank.org/id/ENG00008_2.2.1,https://data.cooperationdatabank.org/id/ENG00008_2.2.2,https://data.cooperationdatabank.org/id/ENG00008_2,https://data.cooperationdatabank.org/id/betweenorwithinparticipantsdesign/1,96.0,48.0,...,-0.032700,0.664078,https://data.cooperationdatabank.org/id/ENG00008,http://dx.doi.org/10.1016/j.jesp.2004.09.004,['https://data.cooperationdatabank.org/vocab/prop/sequentiality'],0.170671,https://data.cooperationdatabank.org/id/sequentiality/simultaneous,https://data.cooperationdatabank.org/id/sequentiality/sequential_turn-taking,https://data.cooperationdata

In [7]:
data_table = pd.DataFrame(columns = ['obs'])
data_table = observations[['obs', 't1', 't2', 'independentProperties']]
data_table

,obs,t1,t2,independentProperties
0,https://data.cooperationdatabank.org/id/ENG00008_1.4.1.2.d,https://data.cooperationdatabank.org/id/ENG00008_1.4.1,https://data.cooperationdatabank.org/id/ENG00008_1.4.2,['https://data.cooperationdatabank.org/vocab/prop/sequentiality']
1,https://data.cooperationdatabank.org/id/ENG00008_1.4.1.3.d,https://data.cooperationdatabank.org/id/ENG00008_1.4.1,https://data.cooperationdatabank.org/id/ENG00008_1.4.3,['https://data.cooperationdatabank.org/vocab/prop/sequentiality']
2,https://data.cooperationdatabank.org/id/ENG00008_1.4.2.3.d,https://data.cooperationdatabank.org/id/ENG00008_1.4.2,https://data.cooperationdatabank.org/id/ENG00008_1.4.3,['https://data.cooperationdatabank.org/vocab/prop/positionInGame']
3,https://data.cooperationdatabank.org/id/ENG00008_2.1.1.2.d,https://data.cooperationdatabank.org/id/ENG00008_2.1.1,https://data.cooperationdatabank.org/id/ENG00008_2.1.2,['https://data.cooperationdatabank.org/vocab/prop/expectationsLevel']
4,https://data.cooperationdatabank.org/id/ENG00008_2.2.1.2.d,https://data.cooperationdatabank.org/id/ENG00008_2.2.1,https://data.cooperationdatabank.org/id/ENG00008_2.2.2,['https://data.cooperationdatabank.org/vocab/prop/sequentiality']
...,...,...,...,...
2475,https://data.cooperationdatabank.org/id/ENG02667_1b.2.2.3.d,https://data.cooperationdatabank.org/id/ENG02667_1b.2.2,https://data.cooperationdatabank.org/id/ENG02667_1b.2.3,['https://data.cooperationdatabank.org/vocab/prop/iteratedPreprogrammedCooperationRate']
2476,https://data.cooperationdatabank.org/id/ENG02668_1.2.1.2.d,https://data.cooperationdatabank.org/id/ENG02668_1.2.1,https://data.cooperationdatabank.org/id/ENG02668_1.2.2,['https://data.cooperationdatabank.org/vocab/prop/sVOType']
2477,https://data.cooperationdatabank.org/id/ENG02668_1.2.1.3.d,https://data.cooperationdatabank.org/id/ENG02668_1.2.1,https://data.cooperationdatabank.org/id/ENG02668_1.2.3,['https://data.cooperationdatabank.org/vocab/prop/sVOType']
2478,https://data.cooperationdatabank.org/id/ENG02668_1.2.2.3.d,https://data.cooperationdatabank.org/id/ENG02668_1.2.2,https://data.cooperationdatabank.org/id/ENG02668_1.2.3,['https://data.cooperationdatabank.org/vocab/prop/sVOType']


In [14]:
def construct_category_data_table(observations): 
    loop = tqdm(total = len(observations), position=0, leave=False)

    categories = pd.DataFrame()
    variables = pd.Series()
    for index, row in observations.iterrows(): 
        obs = row['obs']
        t1 = row['t1']
        t2 = row['t2']
        inds = ast.literal_eval(row['independentProperties'])
        
        for variable in inds:
            variables = variables.append(pd.Series(variable))
            prefix = """ PREFIX owl: <http://www.w3.org/2002/07/owl#>
                    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                    PREFIX class: <https://data.cooperationdatabank.org/vocab/class/>
                    PREFIX property: <https://data.cooperationdatabank.org/vocab/prop/>
                    PREFIX id: <https://data.cooperationdatabank.org/id/> 
                    PREFIX dct: <http://purl.org/dc/terms/>
                    """

            query = prefix + """
                         SELECT * WHERE {
                         """ + '<' + obs + '>' + """ property:treatment ?treatment . 
                         ?treatment """ + '<' + variable + '>' + """ ?categoryOrValue. 
                         """ + '<' + variable + '>' + """  rdfs:range ?range . 

                        }"""

            dfje = get_sparql_dataframe(url, query)
            dfje['IV'] = variable
            categories = categories.append(dfje).reset_index(drop=True)
            
        loop.set_description("Loading...".format(index))
        loop.update(1)
    
    loop.close()
    return categories, variables

categories, variables = construct_category_data_table(data_table)
categories

  0%|                                                                                         | 0/2480 [00:00<?, ?it/s]c:\users\roos\tf\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """


,treatment,categoryOrValue,range,IV
0,https://data.cooperationdatabank.org/id/ENG00008_1.4.1,https://data.cooperationdatabank.org/id/sequentiality/simultaneous,https://data.cooperationdatabank.org/vocab/class/Sequentiality,https://data.cooperationdatabank.org/vocab/prop/sequentiality
1,https://data.cooperationdatabank.org/id/ENG00008_1.4.2,https://data.cooperationdatabank.org/id/sequentiality/sequential_turn-taking,https://data.cooperationdatabank.org/vocab/class/Sequentiality,https://data.cooperationdatabank.org/vocab/prop/sequentiality
2,https://data.cooperationdatabank.org/id/ENG00008_1.4.1,https://data.cooperationdatabank.org/id/sequentiality/simultaneous,https://data.cooperationdatabank.org/vocab/class/Sequentiality,https://data.cooperationdatabank.org/vocab/prop/sequentiality
3,https://data.cooperationdatabank.org/id/ENG00008_1.4.3,https://data.cooperationdatabank.org/id/sequentiality/sequential_turn-taking,https://data.cooperationdatabank.org/vocab/class/Sequentiality,https://data.cooperationdatabank.org/vocab/prop/sequentiality
4,https://data.cooperationdatabank.org/id/ENG00008_1.4.2,1,http://www.w3.org/2001/XMLSchema#integer,https://data.cooperationdatabank.org/vocab/prop/positionInGame
...,...,...,...,...
5083,https://data.cooperationdatabank.org/id/ENG02668_1.2.3,https://data.cooperationdatabank.org/id/svotype/competitor,https://data.cooperationdatabank.org/vocab/class/SVOType,https://data.cooperationdatabank.org/vocab/prop/sVOType
5084,https://data.cooperationdatabank.org/id/ENG02668_1.2.2,https://data.cooperationdatabank.org/id/svotype/individualist,https://data.cooperationdatabank.org/vocab/class/SVOType,https://data.cooperationdatabank.org/vocab/prop/sVOType
5085,https://data.cooperationdatabank.org/id/ENG02668_1.2.3,https://data.cooperationdatabank.org/id/svotype/competitor,https://data.cooperationdatabank.org/vocab/class/SVOType,https://data.cooperationdatabank.org/vocab/prop/sVOType
5086,https://data.cooperationdatabank.org/id/ENG02672_1a.1.1,https://data.cooperationdatabank.org/id/svotype/individualist,https://data.cooperationdatabank.org/vocab/class/SVOType,https://data.cooperationdatabank.org/vocab/prop/sVOType


In [16]:
categories.to_csv('data/categories.csv')

In [17]:
variables.nunique()

128

In [18]:
counts = categories.groupby(['IV', 'categoryOrValue']).size().reset_index(name='Count')
pd.set_option('max_rows', 30)
counts = counts.sort_values('IV')
counts

,IV,categoryOrValue,Count
0,https://data.cooperationdatabank.org/vocab/prop/academicDiscipline,https://data.cooperationdatabank.org/id/academicdiscipline/economics,5
1,https://data.cooperationdatabank.org/vocab/prop/academicDiscipline,https://data.cooperationdatabank.org/id/academicdiscipline/others,6
2,https://data.cooperationdatabank.org/vocab/prop/academicDiscipline,https://data.cooperationdatabank.org/id/academicdiscipline/psychology,1
3,https://data.cooperationdatabank.org/vocab/prop/academicGrade,4,2
4,https://data.cooperationdatabank.org/vocab/prop/academicGrade,6,2
...,...,...,...
534,https://data.cooperationdatabank.org/vocab/prop/uncertaintyLevel,https://data.cooperationdatabank.org/id/uncertaintylevel/low,35
533,https://data.cooperationdatabank.org/vocab/prop/uncertaintyLevel,https://data.cooperationdatabank.org/id/uncertaintylevel/high,34
535,https://data.cooperationdatabank.org/vocab/prop/uncertaintyLevel,https://data.cooperationdatabank.org/id/uncertaintylevel/medium,5
536,https://data.cooperationdatabank.org/vocab/prop/uncertaintyTarget,https://data.cooperationdatabank.org/id/uncertaintytarget/loss,1


In [21]:
counts.to_csv('data/counts.csv')

In [22]:
counts = pd.read_csv('data/counts.csv', index_col = 0)
categories = pd.read_csv('data/categories.csv', index_col = 0)

# Hypotheses Output Functions 

In [24]:
def create_category_output(catT1, catT2, var):
    categories = pd.Series([catT1, catT2])
    categories_placeholder = pd.Series()
    descriptions = pd.Series()
    range_type = query_range(var)
    treatment = pd.Series(['T1', 'T2'])
    
    if range_type != 'http://www.w3.org/2001/XMLSchema#string':
        for index, category in categories.items(): 
            try: 
                description = query_description(category)
            except: 
                continue
            if description != None: 
                slash = category.rfind('/')
                category_str = category[slash+1:]
                category_ser = pd.Series([category_str])
                category_ser.index = [index]
                categories_placeholder = categories_placeholder.append(category_ser)

                description = pd.Series([description])
                description.index = [index]
                descriptions = descriptions.append(description)

            else: 
                description = pd.Series([description])
                descriptions = descriptions.append(description)

    if descriptions.isna().any() == False and lenzi(descriptions) == False: 
        categories = categories_placeholder
        output = pd.DataFrame({'treatment' : treatment,
                               'IV value': categories, 
                               'description': descriptions}).reset_index(drop=True)
    else: 
        output = pd.DataFrame({'treatment' : treatment,
                               'IV value': categories}).reset_index(drop=True)
    return output


def disassemble_statement(statement): 
    result = statement.find(' ')
    item2_3 = statement[result+1:]
    result2 = item2_3.find(' ')

    item1 = statement[:result]
    item1a = item1[:item1.find('_H')]
    item2 = item2_3[:result2].strip()
    item3 = item2_3[result2:].strip()
    item2 = item2.replace('https://data.cooperationdatabank.org/vocab/prop/', '')
    return item1, item1a, item2, item3


def query_label(iri): 
    query = """
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        SELECT * 
        WHERE {
         """ + '<' + iri + '>' + """ rdfs:label ?label . 
        }"""
    dfje = get_sparql_dataframe(url, query)
    try: 
        label = dfje['label'][0]
        return label
    except: 
        print("Except: No label found?")

        
def lenzi(df):
    return len(df.index) == 0


def query_description(entity): 
    prefix = """ PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX class: <https://data.cooperationdatabank.org/vocab/class/>
        PREFIX property: <https://data.cooperationdatabank.org/vocab/prop/>
        PREFIX id: <https://data.cooperationdatabank.org/id/> 
        PREFIX dct: <http://purl.org/dc/terms/>
"""
    query = prefix + """
            SELECT * 
            WHERE {
             """ + '<' + entity + '>' + """ dct:description ?description . 
            }"""
    
    dfje = get_sparql_dataframe(url, query)
   
    if lenzi(dfje) == False: 
        description = dfje['description']
        return description.item()


def effect_string_maker(effect):
    if effect == 'hasPositiveEffectOn': 
        effect_string = 'has positive effect on'
        return effect_string
    elif effect == 'hasNegativeEffectOn': 
        effect_string = 'has negative effect on'
        return effect_string


def query_range(IV): 
    query = """
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        SELECT * 
        WHERE {
         """ + '<' + IV + '>' + """ rdfs:range ?range . 
        }"""
    dfje = get_sparql_dataframe(url, query)

    if dfje.empty == False:
        IVrange = dfje['range'][0]
        return IVrange

    
def control_function(rankings, obs):        
    for index, row in rankings.iterrows(): 
        IV_H, IV, effect, DV = disassemble_statement(row['statement'])
        effect_string = effect_string_maker(effect)
        
        dois = obs[obs['IV_new'].str.contains(str(IV_H))]['doi'].unique().tolist()
        catT1 = obs[obs['IV_new'].str.contains(str(IV_H))]['categoryT1'].unique().tolist()[0]
        catT2 = obs[obs['IV_new'].str.contains(str(IV_H))]['categoryT2'].unique().tolist()[0]
        
        output = create_category_output(catT1, catT2, IV)
        print('\n\nHypothesis ' + str(index+1) + ':', query_label(IV), effect_string, query_label(DV).lower(),
              '\n\nDependent Variable (DV): ', DV, 
              '\n\nIndependent Variable (IV): ', IV, IV_H, 
              '\nDescription IV: ', query_description(IV), 
              '\n\nThe IV categories that were researched and their descriptions are shown below: ')

        with pd.option_context('display.width', 10):  # more options can be specified also
            display(output.style.hide_index())
    
        print('\nThe paper(s) that researched this topic is/are the following: ')
        for doi in dois: 
            if doi == 'nan':
                print('Papers are not available in the data') 
            else: 
                print(doi)

                
#  Most likely examples 
print('POSSIBLE NEW HYPOTHESES WITH HIGHEST PROBABILITIES')
control_function(rankings[:10], observations)


#  least likely examples 
print('\n\nHYPOTHESES WITH VERY LOW PROBABILITIES')
control_function(rankings[len(rankings)-10:], observations)

POSSIBLE NEW HYPOTHESES WITH HIGHEST PROBABILITIES


c:\users\roos\tf\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\roos\tf\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  after removing the cwd from sys.path.




Hypothesis 1: anonymity manipulation has positive effect on cooperation 

Dependent Variable (DV):  https://data.cooperationdatabank.org/id/dependentvariable/cooperation 

Independent Variable (IV):  https://data.cooperationdatabank.org/vocab/prop/anonymityManipul https://data.cooperationdatabank.org/vocab/prop/anonymityManipul_H1 
Description IV:  Degree to which participants' actions were identifiable by their partner(s) and/or the experimenter. 

The IV categories that were researched and their descriptions are shown below: 


treatment,IV value,description
T1,high,The treatment was classified as having a relative high level of anonymity
T2,low,The treatment was classified as having a relative low level of anonymity



The paper(s) that researched this topic is/are the following: 
http://dx.doi.org/10.1080/17565529.2016.1174664
http://dx.doi.org/10.1016/j.jpubeco.2007.11.004
nan
http://dx.doi.org/10.1098/rspb.2017.0689
http://dx.doi.org/10.1016/0167-2681(93)90001-6
http://dx.doi.org/10.1080/00224540209603917
http://dx.doi.org/10.1006/jmps.1997.1203
http://dx.doi.org/10.1093/scan/nst040
http://dx.doi.org/10.1016/j.jebo.2017.07.007
http://dx.doi.org/10.1016/j.geb.2017.08.002
http://dx.doi.org/10.1007/s10683-013-9377-5
http://dx.doi.org/10.1556/jep.11.2013.3.3
http://dx.doi.org/10.1037/h0037450
http://dx.doi.org/10.2139/ssrn.2556325
http://dx.doi.org/10.1016/0167-2681(94)00073-n
http://dx.doi.org/10.1016/S0047-2727(03)00021-5
http://dx.doi.org/10.1177/002200276500900308
http://dx.doi.org/10.1016/j.jesp.2017.02.008
http://dx.doi.org/10.1007/s10683-013-9389-1
http://dx.doi.org/10.1177/1368430209342258
http://dx.doi.org/10.1111/j.2044-8295.2011.02093.x
http://dx.doi.org/10.1037/h0082799


Hypothesis 2: sv

...


The paper(s) that researched this topic is/are the following: 
http://dx.doi.org/10.1016/j.jesp.2008.07.015


Hypothesis 1062: real communication has positive effect on withdrawals 

Dependent Variable (DV):  https://data.cooperationdatabank.org/id/dependentvariable/withdrawals 

Independent Variable (IV):  https://data.cooperationdatabank.org/vocab/prop/communicationReal https://data.cooperationdatabank.org/vocab/prop/communicationReal_H1 
Description IV:  Whether the communication received by participants was real or fake. TRUE = Real communication, FALSE = Fake communication. 

The IV categories that were researched and their descriptions are shown below: 


treatment,IV value
T1,1
T2,0



The paper(s) that researched this topic is/are the following: 
http://dx.doi.org/10.1037//0022-3514.75.3.695


Hypothesis 1063: real communication has negative effect on contributions 

Dependent Variable (DV):  https://data.cooperationdatabank.org/id/dependentvariable/contributions 

Independent Variable (IV):  https://data.cooperationdatabank.org/vocab/prop/communicationReal https://data.cooperationdatabank.org/vocab/prop/communicationReal_H1 
Description IV:  Whether the communication received by participants was real or fake. TRUE = Real communication, FALSE = Fake communication. 

The IV categories that were researched and their descriptions are shown below: 


treatment,IV value
T1,1
T2,0



The paper(s) that researched this topic is/are the following: 
http://dx.doi.org/10.1037//0022-3514.75.3.695


In [26]:
from docx.enum.dml import MSO_THEME_COLOR_INDEX
import docx
def add_hyperlink(paragraph, text, url):
    # This gets access to the document.xml.rels file and gets a new relation id value
    part = paragraph.part
    r_id = part.relate_to(url, docx.opc.constants.RELATIONSHIP_TYPE.HYPERLINK, is_external=True)

    # Create the w:hyperlink tag and add needed values
    hyperlink = docx.oxml.shared.OxmlElement('w:hyperlink')
    hyperlink.set(docx.oxml.shared.qn('r:id'), r_id, )

    # Create a w:r element and a new w:rPr element
    new_run = docx.oxml.shared.OxmlElement('w:r')
    rPr = docx.oxml.shared.OxmlElement('w:rPr')

    # Join all the xml elements together add add the required text to the w:r element
    new_run.append(rPr)
    new_run.text = text
    hyperlink.append(new_run)

    # Create a new Run object and add the hyperlink into it
    r = paragraph.add_run ()
    r._r.append (hyperlink)

    # A workaround for the lack of a hyperlink style (doesn't go purple after using the link)
    # Delete this if using a template that has the hyperlink style in it
    r.font.color.theme_color = MSO_THEME_COLOR_INDEX.HYPERLINK
    r.font.underline = True

    return hyperlink

In [27]:
#  same control function as above, but outputs to a word document 

def control_function2(document, hnumber, rankings, obs):        
    for index, row in rankings.iterrows(): 
        IV_H, IV, effect, DV = disassemble_statement(row['statement'])
        effect_string = effect_string_maker(effect)

        dois = obs[obs['IV_new'].str.contains(str(IV_H))]['doi'].unique().tolist()
        catT1 = obs[obs['IV_new'].str.contains(str(IV_H))]['categoryT1'].unique().tolist()[0]
        catT2 = obs[obs['IV_new'].str.contains(str(IV_H))]['categoryT2'].unique().tolist()[0]
        
        output = create_category_output(catT1, catT2, IV)
        document.add_heading(('Hypothesis ' + str(hnumber) + ': ' + query_label(IV) + ' ' + effect_string + ' ' + query_label(DV).lower()), level=1)
        p = document.add_paragraph('Dependent Variable (DV): ')
        add_hyperlink(p, str(DV), DV)

        p = document.add_paragraph('\nInependent Variable (IV): ')
        add_hyperlink(p, str(IV), IV)
            
        document.add_paragraph('Description IV: ' + query_description(IV) + 
                               '\n\nThe IV values that were researched and (if available) their descriptions are shown below: ')
        
        t = document.add_table(output.shape[0]+1, output.shape[1])
        
        # add the header rows.
        for j in range(output.shape[-1]):
            t.cell(0,j).text = output.columns[j]

        # add the rest of the data frame
        for i in range(output.shape[0]):
            for j in range(output.shape[-1]):
                t.cell(i+1,j).text = str(output.values[i,j])
                
        document.add_paragraph('\nThe paper(s) that researched this topic is/are the following: ')
        for doi in dois: 
            if type(doi) == float:
                continue
            else:
                p = document.add_paragraph('')
                add_hyperlink(p, str(doi), doi)
                
    document.save('hypotheses.docx')

In [28]:
# Create document with the 10 most likely and 10 least likely hypotheses, to use for evaluation 

import random

indexes = [*range(10)] + [*range((len(rankings) - 10), len(rankings))]
# indexes = [0,1,2,3,4,5,6,7,8,9,550,551,552,553,554,555,556,557,558,559]
print(indexes)

def random_index(data):
    choice = random.choice(data) 
    data.remove(choice) 
    return choice, data 

# choice, indexjes = simple(indexes)

document = Document()
document.add_heading('Hypotheses', 0)
hnumber = 0
for r in range(len(indexes)): 
    hnumber += 1
    choice, indexes = random_index(indexes)
    control_function2(document, hnumber, rankings[choice:choice+1], observations)
    print(hnumber, choice)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062]


c:\users\roos\tf\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\roos\tf\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  after removing the cwd from sys.path.


1 2
2 1062
3 1059
4 8
5 1053
6 1055
7 0
8 6
9 9
10 1060
11 5
12 1057
13 7
14 3
15 1
16 1054
17 1061
18 1056
19 4
20 1058
